In [1]:
import sys
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import accuracy_score,roc_curve,auc,precision_score,recall_score

#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
#from expectation_reflection import ExpectationReflection

import expectation_reflection_cv as ERCV
#import expectation_reflection_cvmc as ERCVMC
from function import split_train_test,make_data_balance

In [2]:
np.random.seed(1)

In [3]:
def inference(X_train,y_train,X_test,y_test):     
    n = X_train.shape[1]
    m = len(np.unique(y_train))

    l2 = np.logspace(-5,1,20,base=10.0)
    #l2 = [0.,0.00001,0.0001,0.001,0.01,0.1,1.]
    nl2 = len(l2)
              
    kf = 3   
    kfold = KFold(n_splits=kf,shuffle=False,random_state=1)

    # predict with ER
    #if m == 2:
    h01 = np.zeros(kf)
    w1 = np.zeros((kf,n))
    acc1 = np.zeros(kf)
    
    h0 = np.zeros(nl2)
    w = np.zeros((nl2,n))
    acc = np.zeros(nl2)            
    for il2 in range(nl2):            
        for i,(train_index,val_index) in enumerate(kfold.split(y_train)):
            X_train1, X_val = X_train[train_index], X_train[val_index]
            y_train1, y_val = y_train[train_index], y_train[val_index]
            h01[i],w1[i,:] = ERCV.fit(X_train1,y_train1,X_val,y_val,niter_max=1000,l2=l2[il2])
            
            y_val_pred,p_val_pred = ERCV.predict(X_val,h01[i],w1[i])
            #acc1[i] = accuracy_score(y_val,y_val_pred)
            acc1[i] = ((p_val_pred - y_val)**2).mean()
                                        
        h0[il2] = h01.mean(axis=0)
        w[il2,:] = w1.mean(axis=0)
        acc[il2] = acc1.mean()
    
    il2_select = np.argmin(acc)
    #print('l2:',l2[il2_select])
    y_pred,p_pred = ERCV.predict(X_test,h0[il2_select],w[il2_select,:])

    # entire training set:
    #model = ExpectationReflection(niter_max=1000,l2=l2[il2_select])
    #model.fit(X_train,y_train)
    #y_pred = model.predict(X_test)
                                        
    accuracy = accuracy_score(y_test,y_pred)
    cost = ((p_pred - y_test)**2).mean()
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
  
    # Compute ROC curve and ROC area for each class
    fpr, tpr,thresholds = roc_curve(y_test, p_pred, drop_intermediate=False)
    roc_auc= auc(fpr, tpr)

    #print(len(p_pred),len(y_test),len(fpr),len(tpr))                       

    return accuracy,roc_auc,precision,recall,cost

In [4]:
def compare_inference(X,y,train_size):
    npred = 5
    acc = np.zeros(npred)
    roc_auc = np.zeros(npred)
    precision = np.zeros(npred)
    recall = np.zeros(npred)
    cost = np.zeros(npred)
    for ipred in range(npred):
        X_train,X_test,y_train,y_test = split_train_test(X,y,train_size,test_size=0.7)

        #X_train = MinMaxScaler().fit_transform(X_train)
        #X_test = MinMaxScaler().fit_transform(X_test)
        
        # 2019.07.15
        #acc[ipred],cost[ipred],roc_auc[ipred] = inference(X_train,y_train,X_test,y_test)
        acc[ipred],roc_auc[ipred],precision[ipred],recall[ipred],cost[ipred] \
        = inference(X_train,y_train,X_test,y_test)
            
    #return acc.mean(axis=0),acc.std(),cost.mean(),cost.std(),roc_auc.mean(),roc_auc.std()
    return acc.mean(axis=0),roc_auc.mean(),precision.mean(),recall.mean(),cost.mean(),\
           acc.std(),roc_auc.std(),precision.std(),recall.std(),cost.std()

In [5]:
data_name_list = ['nki','peptide','amputation','wisconsin','heat','hc','paradox','school','stigma','ef',\
                 'smok','coag','anemia','cervix','mental','dement','septic','coimbra','cryo','somerville',\
                  'hcc','heart','liver','language']
n_data = len(data_name_list)

for data_id in range(n_data):
    data_name = data_name_list[data_id]
    # load data
    X = np.loadtxt('data_sets/%s_x.txt'%data_name)
    y = np.loadtxt('data_sets/%s_y.txt'%data_name)
    #print('ini:',np.unique(y,return_counts=True))

    X,y = make_data_balance(X,y)
    #print('balance:',np.unique(y,return_counts=True))

    X, y = shuffle(X, y)
    X = MinMaxScaler().fit_transform(X)                  

    list_train_size = [0.3]
    n_size = len(list_train_size)
    acc = np.zeros((n_size,10)) # 0: acc, 1: acc_std, 2: cost, 3: cost_std
    for i,train_size in enumerate(list_train_size):
        acc[i,:] = compare_inference(X,y,train_size)
        print(i,train_size,acc[i,:])

    np.savetxt('result/%s_acc.dat'%data_name,acc,fmt='%f')

0 0.3 [0.71132075 0.79480242 0.71441925 0.73207547 0.18906471 0.04202086
 0.03536455 0.07002356 0.08128551 0.01777017]
0 0.3 [0.9        0.93515625 0.87421495 0.9375     0.07909754 0.06373774
 0.01686487 0.0591122  0.09682458 0.02496886]
0 0.3 [0.56       0.55644444 0.57571429 0.49333333 0.24931914 0.16918103
 0.17092111 0.18633839 0.17178798 0.01506815]
0 0.3 [0.95405405 0.98791088 0.96306503 0.94459459 0.03378232 0.00921499
 0.00131032 0.01583239 0.00993036 0.00529247]
0 0.3 [0.57966102 0.59747199 0.5916911  0.55932203 0.2488208  0.04535284
 0.03865189 0.05979884 0.06779661 0.01020951]
0 0.3 [0.79428571 0.82881633 0.80209002 0.78857143 0.16892048 0.03331973
 0.02330409 0.04872203 0.07362342 0.02632838]
0 0.3 [0.88636364 0.94783058 0.93045045 0.84090909 0.08709135 0.02032789
 0.01024239 0.05658588 0.04065578 0.01117048]
0 0.3 [0.85744681 0.9118153  0.82007904 0.91914894 0.11788684 0.04070452
 0.00926154 0.04622514 0.03127008 0.01824963]
0 0.3 [0.59339243 0.63742433 0.59081037 0.607413